# Head and Shoulders

In [68]:
import pandas as pd
import pandas._libs.testing as _testing
df = pd.read_csv("EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2022.csv")
df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,time,open,high,low,close,volume
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083
5,05.05.2003 17:00:00.000,1.12718,1.13019,1.12657,1.12804,373668.2930
6,05.05.2003 21:00:00.000,1.12798,1.13004,1.12772,1.12913,94283.7988
7,06.05.2003 01:00:00.000,1.12892,1.12967,1.12743,1.12855,95461.9980
8,06.05.2003 05:00:00.000,1.12856,1.13412,1.12738,1.13381,92809.0996
9,06.05.2003 09:00:00.000,1.13383,1.13662,1.13188,1.13456,90255.7988


In [69]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,15,15), axis=1)
df['shortpivot'] = df.apply(lambda x: pivotid(df, x.name,5,5), axis=1)

In [70]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def shortpointpos(x):
    if x['shortpivot']==1:
        return x['low']-2e-3
    elif x['shortpivot']==2:
        return x['high']+2e-3
    else:
        return np.nan
    
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df['shortpointpos'] = df.apply(lambda row: shortpointpos(row), axis=1)

In [71]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
dfpl = df[7700:7760]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.add_scatter(x=dfpl.index, y=dfpl['shortpointpos'], mode="markers",
                marker=dict(size=5, color="red"),
                name="shortpivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [72]:
from scipy.stats import linregress
backcandles= 14 #!!!should be less than pivot candles

for candleid in range(0, len(df)-backcandles):
    if df.iloc[candleid].pivot != 2 or df.iloc[candleid].shortpivot != 2:
        continue
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    minbcount=0 #minimas before head
    maxbcount=0 #maximas before head
    minacount=0 #minimas after head
    maxacount=0 #maximas after head
    
    for i in range(candleid-backcandles, candleid+backcandles):
        if df.iloc[i].shortpivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name             
            if i < candleid:
                minbcount=+1
            elif i>candleid:
                minacount+=1
        if df.iloc[i].shortpivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
            if i < candleid:
                maxbcount+=1
            elif i>candleid:
                maxacount+=1
    
    if minbcount<1 or minacount<1 or maxbcount<1 or maxacount<1:
        continue

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    headidx = np.argmax(maxim, axis=0)
    if maxim[headidx]-maxim[headidx-1]>1.5e-3 and maxim[headidx]-maxim[headidx+1]>1.5e-3 and abs(slmin)<=1e-4 and xxmin[0]>xxmax[headidx-1] and xxmin[1]<xxmax[headidx+1]:# and (maxim[headidx]-maxim[headidx+1])>(maxim[headidx+1]-minim[headidx+1]) and (maxim[headidx]-maxim[headidx-1])>(maxim[headidx-1]-minim[headidx-1]) :
        print(minbcount,minacount,maxbcount,maxacount, slmin, candleid)
        #print(maxim)
        #print(xxmax)
        #print(minim)
        #print(xxmin)
        break
        
    if candleid % 1000 == 0:
        print(candleid)

1 2 1 1 2.9287833827896943e-05 7731


In [73]:
############################################################

In [74]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta

df['RSI']=ta.rsi(df.close, length=15)
df['EMAF']=ta.ema(df.close, length=20)
df['EMAM']=ta.ema(df.close, length=100)
df['EMAS']=ta.ema(df.close, length=150)

df['TargetNextClose'] = df['close'].shift(-1)

df.dropna(inplace=True)
df.reset_index(inplace = True)
df.drop(['volume', 'close', 'time', 'index', 'pivot', 'shortpivot', 'pointpos', 'shortpointpos'], axis=1, inplace=True)

In [75]:
data_set = df.iloc[0:, 0:10]#.values
pd.set_option('display.max_columns', None)

data_set.head(40)

,open,high,low,RSI,EMAF,EMAM,EMAS,TargetNextClose
0,1.19243,1.19278,1.18619,62.179082,1.180532,1.172327,1.167391,1.18412
1,1.15919,1.16215,1.15173,42.464545,1.156623,1.166813,1.165763,1.15354
2,1.14504,1.14581,1.13974,38.824116,1.146391,1.160703,1.161693,1.14994
3,1.15423,1.16079,1.15261,62.214987,1.149903,1.159695,1.160915,1.15608
4,1.12636,1.13177,1.12587,33.774380,1.138308,1.152074,1.155086,1.13118
5,1.13855,1.13924,1.13700,52.604493,1.135984,1.148306,1.152023,1.13504
6,1.11307,1.11549,1.11123,32.035726,1.126018,1.141134,1.146101,1.11294
7,1.14835,1.15487,1.14439,66.927754,1.143609,1.137926,1.141009,1.15079
8,1.12297,1.12338,1.11348,18.241525,1.132956,1.138165,1.140390,1.11794
9,1.14068,1.14265,1.13922,59.126400,1.134624,1.136598,1.138771,1.13634


In [76]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.2756486  0.27190482 0.27108799 ... 0.23106296 0.21753266 0.25809393]
 [0.21617463 0.21767611 0.20968604 ... 0.22066569 0.21441565 0.20342528]
 [0.19085704 0.18874706 0.18832187 ... 0.20914486 0.20662167 0.19698947]
 ...
 [0.21991412 0.22098685 0.22584726 ... 0.22682302 0.22739181 0.21840642]
 [0.20908928 0.20577873 0.21184206 ... 0.22246765 0.22364304 0.20487334]
 [0.20479513 0.20237948 0.21095114 ... 0.21319584 0.21527202 0.20449792]]


In [82]:
# multiple feature from data provided to the model
X = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 30
print(data_set_scaled.shape[0])
for j in range(7):#data_set_scaled[0].size):#2 columns are target not X
    X.append([])
    for i in range(backcandles, data_set_scaled.shape[0]):#backcandles+2
        X[j].append(data_set_scaled[i-backcandles:i, j])

#move axis from 0 to position 2
X=np.moveaxis(X, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X, yi =np.array(X), np.array(data_set_scaled[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#print(X)
print(X.shape)
#print(y)
print(y.shape)

1203
(1173, 30, 7)
(1173, 1)


In [90]:
class MLP(object):
    """A Multilayer Perceptron class.
    """

    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        """Constructor for the MLP. Takes the number of inputs,
            a variable number of hidden layers, and number of outputs
        Args:
            num_inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        """

        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs

        # create a generic representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]

        # create random connection weights for the layers
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights

        # save derivatives per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives

        # save activations per layer
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations


    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals.
        Args:
            inputs (ndarray): Input signals
        Returns:
            activations (ndarray): Output values
        """

        # the input layer activation is just the input itself
        activations = inputs

        # save the activations for backpropogation
        self.activations[0] = activations

        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)

            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)

            # save the activations for backpropogation
            self.activations[i + 1] = activations

        # return output layer activation
        return activations


    def back_propagate(self, error):
        """Backpropogates an error signal.
        Args:
            error (ndarray): The error to backprop.
        Returns:
            error (ndarray): The final error of the input
        """

        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):

            # get activation for previous layer
            activations = self.activations[i+1]

            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)

            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T

            # get activations for current layer
            current_activations = self.activations[i]

            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)

            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)

            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)


    def train(self, inputs, targets, epochs, learning_rate):
        """Trains model running forward prop and backprop
        Args:
            inputs (ndarray): X
            targets (ndarray): Y
            epochs (int): Num. epochs we want to train the network for
            learning_rate (float): Step to apply to gradient descent
        """
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0

            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]

                # activate the network!
                output = self.forward_propagate(input)

                error = target - output

                self.back_propagate(error)

                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)

                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)

            # Epoch complete, report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))

        print("Training complete!")
        print("=====")


    def gradient_descent(self, learningRate=1):
        """Learns by descending the gradient
        Args:
            learningRate (float): How fast to learn.
        """
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate


    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """

        y = 1.0 / (1 + np.exp(-x))
        return y


    def _sigmoid_derivative(self, x):
        """Sigmoid derivative function
        Args:
            x (float): Value to be processed
        Returns:
            y (float): Output
        """
        return x * (1.0 - x)


    def _mse(self, target, output):
        """Mean Squared Error loss function
        Args:
            target (ndarray): The ground trut
            output (ndarray): The predicted values
        Returns:
            (float): Output
        """
        return np.average((target - output) ** 2)


if __name__ == "__main__":

    # create a dataset to train a network for the sum operation
    items = X[1]
    targets = y

    # create a Multilayer Perceptron with one hidden layer
    mlp = MLP(7, [5], 1)

    # train network
    mlp.train(items, targets, 50, 1)

    # create dummy data
    input = X[1][1]
    target = y[1]

    # get a prediction
    output = mlp.forward_propagate(input)

    print()
    print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))

Error: 0.08477220122159254 at epoch 1
Error: 0.0018997505140680257 at epoch 2
Error: 0.0021945430152002328 at epoch 3
Error: 0.0022013040588154567 at epoch 4
Error: 0.002201472644476457 at epoch 5
Error: 0.0022015033042187765 at epoch 6
Error: 0.0022015310820119754 at epoch 7
Error: 0.002201558798970908 at epoch 8
Error: 0.002201586513989681 at epoch 9
Error: 0.002201614228251789 at epoch 10
Error: 0.002201641941735186 at epoch 11
Error: 0.0022016696543930075 at epoch 12
Error: 0.002201697366178236 at epoch 13
Error: 0.0022017250770442155 at epoch 14
Error: 0.0022017527869446626 at epoch 15
Error: 0.002201780495833662 at epoch 16
Error: 0.0022018082036656655 at epoch 17
Error: 0.002201835910395479 at epoch 18
Error: 0.002201863615978275 at epoch 19
Error: 0.002201891320369575 at epoch 20
Error: 0.0022019190235252587 at epoch 21
Error: 0.002201946725401548 at epoch 22
Error: 0.0022019744259550176 at epoch 23
Error: 0.002202002125142585 at epoch 24
Error: 0.0022020298229215 at epoch 25
E